In [2]:
from pathlib import Path
sql_file = Path("datas\train_gold.sql")

In [3]:
import json

with open('datas\\train.json', 'r') as file:
    text_sql_data = json.load(file)


movie_4_data = []
for movie_4 in text_sql_data:
  if movie_4['db_id'] == "movies_4":
    movie_4_data.append(movie_4)

## Load Model

In [4]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

#MODEL_NAME = "llama3-8b-8192"
MODEL_NAME = "llama3-70b-8192"
#MODEL_NAME  = "mixtral-8x7b-32768"

llm = ChatGroq(model        = MODEL_NAME,
               temperature  = 0,
               api_key      = GROQ_API_KEY)

## Split Data

In [5]:
import random
random.seed(42) 
random.shuffle(movie_4_data)

# Calculate the split indices
total_size  = len(movie_4_data)
train_size  = int(0.7 * total_size)
val_size    = int(0.2 * total_size)

# Split the data
train_data  = movie_4_data[:train_size]
val_data    = movie_4_data[train_size:train_size + val_size]
test_data   = movie_4_data[train_size + val_size:]

# Results
print(f"Train size: {len(train_data)}")
print(f"Validation size: {len(val_data)}")
print(f"Test size: {len(test_data)}")

Train size: 110
Validation size: 31
Test size: 17


In [6]:
import random
random.seed(42) 
random.shuffle(movie_4_data)

# Calculate the split indices
total_size  = len(movie_4_data)
train_size  = int(0.7 * total_size)

# Split the data
train_data  = movie_4_data[:train_size]
test_data   = movie_4_data[train_size:]

# Results
print(f"Train size: {len(train_data)}")
print(f"Test size: {len(test_data)}")

Train size: 110
Test size: 48


## Batabase connection

In [7]:
import sqlite3
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

sqlite_file_path = "datas\\movies_4.sqlite"

conn    = sqlite3.connect(sqlite_file_path)
cursor  = conn.cursor()

copy_train_data = train_data.copy()

for data in copy_train_data:
  query = data['SQL']
  cursor.execute(query)

  results = cursor.fetchall()
  data["Answer"] = ",".join([str(ans[0]) for ans in results])

conn.close()



conn  = create_engine(f"sqlite:///{sqlite_file_path}")
db    = SQLDatabase(conn)

## Alternating the Data

In [8]:
SQL_data = []

for item in copy_train_data:
    new_item = {
        "Question"  : f"{item['question']}==>> {item['evidence']}",
        "SQLQuery"  : item['SQL'],
        "SQLResult" : "Result of the SQL query",  # Placeholder for the actual SQL result if needed
        "Answer"    : item['Answer']
    }
    SQL_data.append(new_item)

## Few_shot data

In [9]:
import random
few_shots = random.sample(SQL_data,12)

## Creating Semantic Similarity Based example selector

In [10]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma



embeddings    = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize  = [' '.join(sent.values()) for sent in few_shots]                                   #[' '.join([str(item) for value in data.values() for item in (value if isinstance(value, list) else [value])])for data in few_shots]
vectorstore   = Chroma.from_texts(to_vectorize,embeddings,metadatas=few_shots)

example_selector = SemanticSimilarityExampleSelector(vectorstore  = vectorstore,k= 2,)


C:\Users\Asus\AppData\Local\Temp\ipykernel_45440\1261799606.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings    = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Asus\anaconda3\envs\llm_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Asus\anaconda3\envs\llm_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by defa

In [11]:
example_selector.select_examples({'question': 'What keyword can the user use to search for the movie Finding Nemo?',})

[{'Answer': "City of Angels,It's a Wonderful Life,Dogma,The Prophecy,Frailty,Legion,The Mortal Instruments: City of Bones,The Christmas Candle",
  'Question': 'Look for the movie title with the keyword of "angel".==>> keyword of "angel" refers to keyword_name = \'angel\'',
  'SQLQuery': "SELECT T1.title FROM movie AS T1 INNER JOIN movie_keywords AS T2 ON T1.movie_id = T2.movie_id INNER JOIN keyword AS T3 ON T2.keyword_id = T3.keyword_id WHERE T3.keyword_name = 'angel'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'Iron Man 3',
  'Question': "Which movie has the keyword 'extremis?'==>> Which movie refers to title; keyword 'extremis' refers to keyword_name = 'extremis'",
  'SQLQuery': "SELECT T1.title FROM movie AS T1 INNER JOIN movie_keywords AS T2 ON T1.movie_id = T2.movie_id INNER JOIN keyword AS T3 ON T2.keyword_id = T3.keyword_id WHERE T3.keyword_name = 'extremis'",
  'SQLResult': 'Result of the SQL query'}]

## Prompting

In [12]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX
from langchain.prompts.prompt import PromptTemplate
from langchain_experimental.sql import SQLDatabaseChain

In [13]:
SQLite_prompt = """
You are a SQLite expert. Given an input question , first create a syntactically correct SQLite query (standard sql query and do not include any extra symbals) to run,
then look at the results of the query and return the answer only. 
If relevant evidence is provided, use it to infer additional conditions or filters in the query.
For example, if the evidence states "The condition of the loans requires the account type should be the owner," you can infer the condition "account.type = 'OWNER'" in the query.

Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite.
You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question.
Wrap each column name in double quotes (") to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist.Only use the column names listed in the tables below.
Also, pay attention to which column is in which table.


If the question involves "today," use the `date('now')` function to get the current date.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run 
SQLResult: Result of the SQLQuery
Answer: Final answer here


Only respond in JSON format, dont include anything other than json in output.
"""

In [14]:
"""(with evidence provided after the question, ending with ==>>)"""

'(with evidence provided after the question, ending with ==>>)'

In [15]:

example_prompt = PromptTemplate(
    input_variables =['Question','SQLQuery','SQLResult','Answer',],
    template        ='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}',
)

few_shot_prompt = FewShotPromptTemplate(example_selector= example_selector,
                                        example_prompt  = example_prompt,
                                        prefix          = SQLite_prompt,
                                        suffix          = PROMPT_SUFFIX,
                                        input_variables=['input', 'table_info', 'top_k'])

In [16]:
PROMPT_SUFFIX

'Only use the following tables:\n{table_info}\n\nQuestion: {input}'

## Chain 

In [17]:
from langchain.chains import create_sql_query_chain
new_chain = create_sql_query_chain(llm=llm,db=db,prompt=few_shot_prompt)

In [42]:
ques = {'question': "When was the release date of the latest movie in which Dariusz Wolski worked as a crew member? ==>> release date of the latest movie refers to max(release_date)" }
ques_1 = {"question":
         "What is the average revenue of the movie in which Dariusz Wolski works as the director of photography? ==>> director of photography refers to job = 'Director of Photography'; average revenue = divide(sum(revenue), count(movie_id))" }
ques_2 = {'question': 'Provide the release date and language of the most popular movie.==>> language refers to langauge_name; most popular movie refers to max(popularity)'}

In [43]:
new_chain.invoke(ques_2)

'{\n"Question": "Provide the release date and language of the most popular movie.",\n"SQLQuery": "SELECT T1.release_date, T3.language_name FROM movie AS T1 INNER JOIN movie_languages AS T2 ON T1.movie_id = T2.movie_id INNER JOIN language AS T3 ON T2.language_id = T3.language_id ORDER BY T1.popularity DESC LIMIT 1",\n"SQLResult": "Result of the SQL query",\n"Answer": "2003-05-30, English"\n}'

In [44]:
output = new_chain.invoke(ques_2) 
print(output)

{
"Question": "Provide the release date and language of the most popular movie.",
"SQLQuery": "SELECT T1.release_date, T3.language_name FROM movie AS T1 INNER JOIN movie_languages AS T2 ON T1.movie_id = T2.movie_id INNER JOIN language AS T3 ON T2.language_id = T3.language_id ORDER BY T1.popularity DESC LIMIT 1",
"SQLResult": "Result of the SQL query",
"Answer": "2003-05-30, English"
}


In [38]:
output

'{\n"Question": "Provide the release date and language of the most popular movie.",\n"SQLQuery": "SELECT T1.release_date, T3.language_name FROM movie AS T1 INNER JOIN movie_languages AS T2 ON T1.movie_id = T2.movie_id INNER JOIN language AS T3 ON T2.language_id = T3.language_id ORDER BY T1.popularity DESC LIMIT 1",\n"SQLResult": "Result of the SQL query",\n"Answer": "2003-05-30, English"\n}'

In [39]:
output = json.loads(output)
output

{'Question': 'Provide the release date and language of the most popular movie.',
 'SQLQuery': 'SELECT T1.release_date, T3.language_name FROM movie AS T1 INNER JOIN movie_languages AS T2 ON T1.movie_id = T2.movie_id INNER JOIN language AS T3 ON T2.language_id = T3.language_id ORDER BY T1.popularity DESC LIMIT 1',
 'SQLResult': 'Result of the SQL query',
 'Answer': '2003-05-30, English'}

In [40]:
#qu = output.split("\n\n")[1].split("SQLQuery:")[-1]
qu =output['SQLQuery']
qu

'SELECT T1.release_date, T3.language_name FROM movie AS T1 INNER JOIN movie_languages AS T2 ON T1.movie_id = T2.movie_id INNER JOIN language AS T3 ON T2.language_id = T3.language_id ORDER BY T1.popularity DESC LIMIT 1'

In [24]:
db.run(val_data[15]['SQL'])

"[('2015-09-30',)]"

In [41]:
db.run(qu)

"[('2015-06-17', 'English')]"

In [26]:
val_data[22]

{'db_id': 'movies_4',
 'question': 'What keyword can the user use to search for the movie Finding Nemo?',
 'evidence': "What keyword refers to keyword_name; Finding Nemo refers to title = 'Finding Nemo'",
 'SQL': "SELECT T3.keyword_name FROM movie AS T1 INNER JOIN movie_keywords AS T2 ON T1.movie_id = T2.movie_id INNER JOIN keyword AS T3 ON T2.keyword_id = T3.keyword_id WHERE T1.title = 'Finding Nemo'"}

In [34]:
train_data[19]

{'db_id': 'movies_4',
 'question': 'Provide the release date and language of the most popular movie.',
 'evidence': 'language refers to langauge_name; most popular movie refers to max(popularity)',
 'SQL': 'SELECT T1.release_date, T3.language_name FROM movie AS T1 INNER JOIN movie_languages AS T2 ON T1.movie_id = T2.movie_id INNER JOIN language AS T3 ON T2.language_id = T3.language_id ORDER BY T1.popularity DESC LIMIT 1',
 'Answer': '2015-06-17'}